In [4]:
# export_drone_wavs_bytes.py
# Write WAVs straight from Parquet without using torchcodec.
import os, glob
from datasets import load_dataset, Audio
from tqdm import tqdm

# ---- paths ----
DATASET_PATH = r"D:/FYP/Datasets/Drone Datasets/drone-audio-detection-samples hf/data"
OUTPUT_ROOT  = r"D:/FYP/Datasets/Drone Datasets/drone-audio-detection-samples hf/extracted_data"
# ---------------

os.makedirs(OUTPUT_ROOT, exist_ok=True)
out_drone   = os.path.join(OUTPUT_ROOT, "drone")
out_nodrone = os.path.join(OUTPUT_ROOT, "nodrone")
os.makedirs(out_drone, exist_ok=True)
os.makedirs(out_nodrone, exist_ok=True)

# Find all parquet shards
parquet_files = sorted(glob.glob(os.path.join(DATASET_PATH, "**", "*.parquet"), recursive=True))
if not parquet_files:
    raise SystemExit(f"No .parquet files found under: {DATASET_PATH}")
print(f"Found {len(parquet_files)} parquet shards.")

total_exported = 0

for shard_idx, parquet_path in enumerate(parquet_files):
    # Load one shard; IMPORTANT: decode=False to avoid audio backend entirely
    ds = load_dataset("parquet", data_files=parquet_path, split="train")
    ds = ds.cast_column("audio", Audio(decode=False))

    pbar = tqdm(ds, desc=f"Shard {shard_idx+1:02d}/{len(parquet_files)}", unit="row")
    for row_idx, row in enumerate(pbar):
        try:
            audio = row["audio"]      # dict with possibly 'bytes' and/or 'path'
            label = row["label"]      # 0 = no-drone, 1 = drone

            # Where to save
            if label == 1:
                out_dir = out_drone
                prefix = "drone"
            else:
                out_dir = out_nodrone
                prefix = "nodrone"

            fname = f"{prefix}_shard{shard_idx:02d}_idx{row_idx:06d}.wav"
            fpath = os.path.join(out_dir, fname)

            # Skip if already written (safe resume)
            if os.path.exists(fpath):
                continue

            # Prefer raw bytes (fast, no decoding backend needed)
            if isinstance(audio, dict) and audio.get("bytes") is not None:
                with open(fpath, "wb") as f:
                    f.write(audio["bytes"])
                total_exported += 1
                continue

            # If HF stored a temp file path, copy it
            if isinstance(audio, dict) and audio.get("path"):
                # Fall back to copying the file contents
                with open(audio["path"], "rb") as src, open(fpath, "wb") as dst:
                    dst.write(src.read())
                total_exported += 1
                continue

            # If neither bytes nor path, log and continue
            pbar.write(f"Row {row_idx}: no 'bytes' or 'path' in audio; skipping.")
        except Exception as e:
            pbar.write(f"Skipping row {row_idx} in shard {shard_idx}: {e}")

print(f"Done. Exported {total_exported} WAV files into:\n- {out_drone}\n- {out_nodrone}")


Found 39 parquet shards.


Shard 01/39: 100%|██████████| 4624/4624 [00:15<00:00, 292.99row/s] 


Generating train split: 0 examples [00:00, ? examples/s]

Shard 02/39: 100%|██████████| 4624/4624 [00:19<00:00, 238.35row/s]


Generating train split: 0 examples [00:00, ? examples/s]

...

Generating train split: 0 examples [00:00, ? examples/s]

Shard 38/39: 100%|██████████| 4623/4623 [00:03<00:00, 1445.72row/s]


Generating train split: 0 examples [00:00, ? examples/s]

Shard 39/39: 100%|██████████| 4623/4623 [00:02<00:00, 1627.63row/s]

Done. Exported 180320 WAV files into:
- D:/FYP/Datasets/Drone Datasets/drone-audio-detection-samples hf/extracted_data\drone
- D:/FYP/Datasets/Drone Datasets/drone-audio-detection-samples hf/extracted_data\nodrone
